1-Generate the Dataset: We'll create a dataset with 50-100 sentences for training and 10 sentences for each of the 10 actions.

2-Preprocess the Text: This involves removing punctuation and tokenizing the text using Python's string library.

3-Build the Keras Model: We'll create a Keras model with an embedding layer, LSTM layers, and a softmax activation function for classification.

In [16]:
import random
import string
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pickle



In [17]:
actions = {
    1: "open the door",
    2: "close the door",
    3: "turn on the light",
    4: "turn off the light",
    5: "turn on the AC",
    6: "turn off the AC",
    7: "open the window",
    8: "close the window",
    9: "turn on the heater",
    10: "turn off the heater"
}

sentences = {
    1: [
        "Please open the door", "Can you open the door?", "I need the door to be opened", "Open the door, please",
        "Would you open the door?", "Door should be opened", "Could you open the door?", "I want the door open",
        "The door needs to be opened", "Open the door now", "Could you kindly open the door?", "The door must be opened",
        "Opening the door would be great", "Please, can you open the door?", "I'd appreciate it if you open the door",
        "Can you please open the door?", "Make sure to open the door", "The door has to be opened", "It's necessary to open the door",
        "Do open the door", "Opening the door is needed", "I'd like you to open the door", "Would you kindly open the door?",
        "Please ensure the door is opened", "Opening the door is required", "You should open the door", "I request you to open the door",
        "Be sure to open the door", "Would you do me the favor of opening the door?", "I insist you open the door"
    ],
    2: [
        "Please close the door", "Can you close the door?", "I need the door to be closed", "Close the door, please",
        "Would you close the door?", "Door should be closed", "Could you close the door?", "I want the door closed",
        "The door needs to be closed", "Close the door now", "Could you kindly close the door?", "The door must be closed",
        "Closing the door would be great", "Please, can you close the door?", "I'd appreciate it if you close the door",
        "Can you please close the door?", "Make sure to close the door", "The door has to be closed", "It's necessary to close the door",
        "Do close the door", "Closing the door is needed", "I'd like you to close the door", "Would you kindly close the door?",
        "Please ensure the door is closed", "Closing the door is required", "You should close the door", "I request you to close the door",
        "Be sure to close the door", "Would you do me the favor of closing the door?", "I insist you close the door"
    ],
    3: [
        "Please turn on the light", "Can you turn on the light?", "I need the light on", "Turn on the light, please",
        "Would you turn on the light?", "Light should be turned on", "Could you turn on the light?", "I want the light on",
        "The light needs to be turned on", "Turn on the light now", "Could you kindly turn on the light?", "The light must be turned on",
        "Turning on the light would be great", "Please, can you turn on the light?", "I'd appreciate it if you turn on the light",
        "Can you please turn on the light?", "Make sure to turn on the light", "The light has to be turned on", "It's necessary to turn on the light",
        "Do turn on the light", "Turning on the light is needed", "I'd like you to turn on the light", "Would you kindly turn on the light?",
        "Please ensure the light is turned on", "Turning on the light is required", "You should turn on the light", "I request you to turn on the light",
        "Be sure to turn on the light", "Would you do me the favor of turning on the light?", "I insist you turn on the light"
    ],
    4: [
        "Please turn off the light", "Can you turn off the light?", "I need the light off", "Turn off the light, please",
        "Would you turn off the light?", "Light should be turned off", "Could you turn off the light?", "I want the light off",
        "The light needs to be turned off", "Turn off the light now", "Could you kindly turn off the light?", "The light must be turned off",
        "Turning off the light would be great", "Please, can you turn off the light?", "I'd appreciate it if you turn off the light",
        "Can you please turn off the light?", "Make sure to turn off the light", "The light has to be turned off", "It's necessary to turn off the light",
        "Do turn off the light", "Turning off the light is needed", "I'd like you to turn off the light", "Would you kindly turn off the light?",
        "Please ensure the light is turned off", "Turning off the light is required", "You should turn off the light", "I request you to turn off the light",
        "Be sure to turn off the light", "Would you do me the favor of turning off the light?", "I insist you turn off the light"
    ],
    5: [
        "Please turn on the AC", "Can you turn on the AC?", "I need the AC on", "Turn on the AC, please",
        "Would you turn on the AC?", "AC should be turned on", "Could you turn on the AC?", "I want the AC on",
        "The AC needs to be turned on", "Turn on the AC now", "Could you kindly turn on the AC?", "The AC must be turned on",
        "Turning on the AC would be great", "Please, can you turn on the AC?", "I'd appreciate it if you turn on the AC",
        "Can you please turn on the AC?", "Make sure to turn on the AC", "The AC has to be turned on", "It's necessary to turn on the AC",
        "Do turn on the AC", "Turning on the AC is needed", "I'd like you to turn on the AC", "Would you kindly turn on the AC?",
        "Please ensure the AC is turned on", "Turning on the AC is required", "You should turn on the AC", "I request you to turn on the AC",
        "Be sure to turn on the AC", "Would you do me the favor of turning on the AC?", "I insist you turn on the AC"
    ],
    6: [
        "Please turn off the AC", "Can you turn off the AC?", "I need the AC off", "Turn off the AC, please",
        "Would you turn off the AC?", "AC should be turned off", "Could you turn off the AC?", "I want the AC off",
        "The AC needs to be turned off", "Turn off the AC now", "Could you kindly turn off the AC?", "The AC must be turned off",
        "Turning off the AC would be great", "Please, can you turn off the AC?", "I'd appreciate it if you turn off the AC",
        "Can you please turn off the AC?", "Make sure to turn off the AC", "The AC has to be turned off", "It's necessary to turn off the AC",
        "Do turn off the AC", "Turning off the AC is needed", "I'd like you to turn off the AC", "Would you kindly turn off the AC?",
        "Please ensure the AC is turned off", "Turning off the AC is required", "You should turn off the AC", "I request you to turn off the AC",
        "Be sure to turn off the AC", "Would you do me the favor of turning off the AC?", "I insist you turn off the AC"
    ],
    7: [
        "Please open the window", "Can you open the window?", "I need the window to be opened", "Open the window, please",
        "Would you open the window?", "Window should be opened", "Could you open the window?", "I want the window open",
        "The window needs to be opened", "Open the window now", "Could you kindly open the window?", "The window must be opened",
        "Opening the window would be great", "Please, can you open the window?", "I'd appreciate it if you open the window",
        "Can you please open the window?", "Make sure to open the window", "The window has to be opened", "It's necessary to open the window",
        "Do open the window", "Opening the window is needed", "I'd like you to open the window", "Would you kindly open the window?",
        "Please ensure the window is opened", "Opening the window is required", "You should open the window", "I request you to open the window",
        "Be sure to open the window", "Would you do me the favor of opening the window?", "I insist you open the window"
    ],
    8: [
        "Please close the window", "Can you close the window?", "I need the window to be closed", "Close the window, please",
        "Would you close the window?", "Window should be closed", "Could you close the window?", "I want the window closed",
        "The window needs to be closed", "Close the window now", "Could you kindly close the window?", "The window must be closed",
        "Closing the window would be great", "Please, can you close the window?", "I'd appreciate it if you close the window",
        "Can you please close the window?", "Make sure to close the window", "The window has to be closed", "It's necessary to close the window",
        "Do close the window", "Closing the window is needed", "I'd like you to close the window", "Would you kindly close the window?",
        "Please ensure the window is closed", "Closing the window is required", "You should close the window", "I request you to close the window",
        "Be sure to close the window", "Would you do me the favor of closing the window?", "I insist you close the window"
    ],
    9: [
        "Please turn on the heater", "Can you turn on the heater?", "I need the heater on", "Turn on the heater, please",
        "Would you turn on the heater?", "Heater should be turned on", "Could you turn on the heater?", "I want the heater on",
        "The heater needs to be turned on", "Turn on the heater now", "Could you kindly turn on the heater?", "The heater must be turned on",
        "Turning on the heater would be great", "Please, can you turn on the heater?", "I'd appreciate it if you turn on the heater",
        "Can you please turn on the heater?", "Make sure to turn on the heater", "The heater has to be turned on", "It's necessary to turn on the heater",
        "Do turn on the heater", "Turning on the heater is needed", "I'd like you to turn on the heater", "Would you kindly turn on the heater?",
        "Please ensure the heater is turned on", "Turning on the heater is required", "You should turn on the heater", "I request you to turn on the heater",
        "Be sure to turn on the heater", "Would you do me the favor of turning on the heater?", "I insist you turn on the heater"
    ],
    10: [
        "Please turn off the heater", "Can you turn off the heater?", "I need the heater off", "Turn off the heater, please",
        "Would you turn off the heater?", "Heater should be turned off", "Could you turn off the heater?", "I want the heater off",
        "The heater needs to be turned off", "Turn off the heater now", "Could you kindly turn off the heater?", "The heater must be turned off",
        "Turning off the heater would be great", "Please, can you turn off the heater?", "I'd appreciate it if you turn off the heater",
        "Can you please turn off the heater?", "Make sure to turn off the heater", "The heater has to be turned off", "It's necessary to turn off the heater",
        "Do turn off the heater", "Turning off the heater is needed", "I'd like you to turn off the heater", "Would you kindly turn off the heater?",
        "Please ensure the heater is turned off", "Turning off the heater is required", "You should turn off the heater", "I request you to turn off the heater",
        "Be sure to turn off the heater", "Would you do me the favor of turning off the heater?", "I insist you turn off the heater"
    ]
}



In [18]:
data = []
for action, sents in sentences.items():
    for sent in sents:
        data.append((sent, action))

random.shuffle(data)

# Function to preprocess text
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

# Apply preprocessing
texts = [preprocess_text(text) for text, label in data]
labels = [label for text, label in data]

# Tokenize texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences
max_length = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_length, padding='post')

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
y = np.eye(len(actions))[y]  # One-hot encoding

# Save tokenizer and label encoder
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('label_encoder.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [19]:
# Build the model
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
embedding_dim = 50

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(len(actions), activation='softmax'))

In [20]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=73, batch_size=32, validation_split=0.2)
model.save('My_Siri.h5')






Epoch 1/73
8/8 [==============================] - 26s 3s/step - loss: 2.3048 - accuracy: 0.0875 - val_loss: 2.3034 - val_accuracy: 0.0833
Epoch 2/73
8/8 [==============================] - 21s 3s/step - loss: 2.2892 - accuracy: 0.1917 - val_loss: 2.2617 - val_accuracy: 0.1667
Epoch 3/73
8/8 [==============================] - 19s 2s/step - loss: 2.1086 - accuracy: 0.2333 - val_loss: 1.8917 - val_accuracy: 0.1667
Epoch 4/73
8/8 [==============================] - 18s 2s/step - loss: 1.8325 - accuracy: 0.1833 - val_loss: 1.7635 - val_accuracy: 0.2000
Epoch 5/73
8/8 [==============================] - 21s 2s/step - loss: 1.7702 - accuracy: 0.1875 - val_loss: 1.7165 - val_accuracy: 0.1667
Epoch 6/73
8/8 [==============================] - 19s 2s/step - loss: 1.7097 - accuracy: 0.2375 - val_loss: 1.6163 - val_accuracy: 0.2667
Epoch 7/73
8/8 [==============================] - 17s 2s/step - loss: 1.5923 - accuracy: 0.3417 - val_loss: 1.4913 - val_accuracy: 0.3500
Epoch 8/73
8/8 [==================

In [21]:
import speech_recognition as sr


# Load the pre-trained model
model = load_model('My_Siri.h5')

# Function to load tokenizer
def load_tokenizer(filepath):
    with open(filepath, 'rb') as file:
        tokenizer = pickle.load(file)
    return tokenizer

# Function to load label encoder
def load_label_encoder(filepath):
    with open(filepath, 'rb') as file:
        label_encoder = pickle.load(file)
    return label_encoder

# Load tokenizer and label encoder
tokenizer = load_tokenizer('tokenizer.pickle')
label_encoder = load_label_encoder('label_encoder.pickle')

# Function to recognize voice
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Say something...")
        audio = recognizer.listen(source)

    try:
        print("Recognizing...")
        text = recognizer.recognize_google(audio)
        print("Speech recognized:", text)
        return text
    except sr.UnknownValueError:
        print("Sorry, could not understand audio.")
        return ""
    except sr.RequestError as e:
        print("Error fetching results; {0}".format(e))
        return ""

# Process voice input and make predictions
def process_voice_input():
    text = recognize_speech()
    if text:
        preprocessed_text = preprocess_text(text)
        sequence = tokenizer.texts_to_sequences([preprocessed_text])
        padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')
        prediction = model.predict(padded_sequence)
        predicted_label = np.argmax(prediction)
        predicted_action = label_encoder.inverse_transform([predicted_label])[0]
        print("Predicted Action:", predicted_action, "(", actions[predicted_action], ")")


In [34]:
# Run the process
process_voice_input()

Say something...
Recognizing...
Speech recognized: AC should be off AC should be off
1/1 [==============================] - 0s 19ms/step
Predicted Action: 6 ( turn off the AC )
